In [4]:
import pandas as pd
 
#with pd.HDFStore('myfile.h5', 'r') as d:
#    df = d.get('TheData')
#    df.to_csv('myfile.csv')

In [5]:
import numpy as np
import h5py

In [26]:
class NetFly(object):
    def __init__(self,flynum,rootpath = '/media/imager/FlyDataD/FlyDB/'):
        self.flynum = flynum
        self.flypath = rootpath + '/Fly%3d'%flynum
        self.flatpaths = [os.path.join(dp, f) for dp, dn, fn in os.walk(self.flypath) for f in fn]
        try:
            self.h5paths = [fn for fn in self.flatpaths if fn[-4:] == 'hdf5']
        except(IndexError):
            pass
        try:
            self.txtpaths = [fn for fn in self.flatpaths if fn[-3:] == 'txt']
        except(IndexError):
            pass
        try:
            self.pklpaths = [fn for fn in self.flatpaths if fn[-4:] == 'cpkl']
        except(IndexError):
            pass
        try:
            self.script_name = [x for x in os.listdir(self.flypath) if '.py' in x][0].split('.py')[0]
        except(IndexError):
            pass
        try:
            self.tiffpaths = [fn for fn in self.flatpaths if fn[-4:] == 'tif']
        except(IndexError):
            pass
        try:
            self.pngpaths = [fn for fn in self.flatpaths if fn[-3:] == 'png']
        except(IndexError):
            pass
        try:
            self.bagpaths = [fn for fn in self.flatpaths if fn[-3:] == 'bag']
        except(IndexError):
            pass
        try:
            self.abfpaths = [fn for fn in self.flatpaths if fn[-3:] == 'abf']
        except(IndexError):
            pass
        
    def open_signals(self,extensions = ['hdf5','txt','cpkl','tif','png','bag'],verbose = False):
        self.h5files = dict()
        if not(type(extensions) is list):
            extensions = [extensions]
        if 'hdf5' in extensions:
            if verbose: print('opening hdf5')
            for fn in self.h5paths:
                key = fn.split('/')[-1].split('.')[0]
                self.h5files[key] = h5py.File(fn,'r')
                self.__dict__[key] = self.h5files[key][key]
        if 'txt' in extensions:
            if verbose: print('opening txt')
            for fn in self.txtpaths:
                key = fn.split('/')[-1].split('.')[0]
                with open(fn,'rt') as f:
                    self.__dict__[key] = f.readlines()
        if 'cpkl' in extensions:
            import cPickle
            if verbose: print('opening pkl')
            for fn in self.pklpaths:
                key = fn.split('/')[-1].split('.')[0]
                with open(fn,'rb') as f:
                    self.__dict__[key] = cPickle.load(f)
        if 'tif' in extensions:
            from skimage import io
            for fn in self.tiffpaths:
                key = fn.splsit('/')[-1].split('.')[0]
                self.__dict__[key] = io.imread(fn)
        if 'png' in extensions:
            from skimage import io
            for fn in self.pngpaths:
                key = fn.split('/')[-1].split('.')[0]
                self.__dict__[key] = io.imread(fn)
    
    def save_pickle(self,data,filename):
        import cPickle
        if filename.split('.')[-1] == 'cpkl':
            with open(os.path.join(self.flypath,filename),'wb') as f:
                cPickle.dump(data,f)
            self.__dict__[filename.split('.')[0]] = data
        else:
            with open(os.path.join(self.flypath,filename + '.cpkl'),'wb') as f:
                cPickle.dump(data,f)
            self.__dict__[filename] = data
            
    def save_hdf5(self,data,filename,overwrite = False):
        import h5py
        if filename.split('.')[-1] == 'hdf5':
            fn = os.path.join(self.flypath,filename)
            if os.path.exists(fn) and overwrite:
                os.remove(fn)
            h_file = h5py.File(fn)
            h_file.create_dataset(filename,data = data,compression = 'gzip')
            h_file.flush()
            self.__dict__[filename.split('.')[0]] = data
        else:
            fn = os.path.join(self.flypath,filename + '.hdf5')
            if os.path.exists(fn) and overwrite:
                os.remove(fn)
            h_file = h5py.File(fn)
            h_file.create_dataset(filename,data = data,compression = 'gzip')
            h_file.flush()
            self.__dict__[filename] = data
    
    def save_txt(self,string,filename):
        if filename.split('.')[-1] == 'txt':
            fn = os.path.join(self.flypath,filename)
            with open(os.path.join(self.flypath,filename),'wb') as f:
                f.write(string)
            self.__dict__[filename.split('.')[0]] = string
        else:
            with open(os.path.join(self.flypath,filename + '.txt'),'wb') as f:
                f.write(string)
            self.__dict__[filename] = string

    def save_py(self,string,filename):
        if filename.split('.')[-1] == 'py':
            fn = os.path.join(self.flypath,filename)
            with open(os.path.join(self.flypath,filename),'wb') as f:
                f.write(string)
            self.__dict__[filename.split('.')[0]] = string
        else:
            with open(os.path.join(self.flypath,filename + '.py'),'wb') as f:
                f.write(string)
            self.__dict__[filename] = string

    def run_py(self,filename,pass_flynum = False):
        import subprocess
        if pass_flynum:
            return subprocess.Popen(['python',
                                     os.path.join(self.flypath,filename),
                                     str(self.flynum)])
        else:
            return subprocess.Popen(['python',os.path.join(self.flypath,filename)])
        
    def copy_to_flydir(self,filename):
        import shutil
        shutil.copy(filename,self.flypath)
        
    def close_signals(self):
        for f in self.h5files.values():
            f.close()

    def get_last_git_comment(self):
        "return a dictionary with the last comments for the repos tracked in the git_SHA.txt file"
        import os
        if os.path.join(self.flypath,'git_SHA.txt') in self.txtpaths:
            self.open_signals(extensions = ['txt'])
            cmnd_strs = ["git -C '%s' show --no-patch --oneline %s"%tuple(sh.split(':')) for sh in self.git_SHA]
            cmmt_strs = [(r.split(':')[0].split('/')[-1],os.popen(s.strip()).read()) for r,s in zip(self.git_SHA,cmnd_strs)]
            cmmt_dict = dict()
            for k,v in cmmt_strs:
                cmmt_dict[k] = v
            return cmmt_dict
        else:
            import warnings
            warnings.warn('Fly%s does not contain a git_SHA.txt file'%(self.flynum))
            return {}

IndentationError: unindent does not match any outer indentation level (<ipython-input-26-b3bbe30498f2>, line 35)

In [30]:
import h5py
filename = "kappa_debug_log.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

Keys: [u'frenet_log', u'kappa_log']


In [31]:
data

[u'axis0',
 u'axis1',
 u'block0_items',
 u'block0_values',
 u'block1_items',
 u'block1_values']

In [33]:
n1[0].value

AttributeError: 'numpy.unicode_' object has no attribute 'value'

In [28]:
hf = h5py.File('kappa_debug_log.h5', 'r')

In [29]:
hf.keys()

[u'frenet_log', u'kappa_log']

In [4]:
n1 = hf.get('frenet_log')
n2 = hf.get('kappa_log')

In [5]:
n1 = np.array(n1)
n1.shape

(6,)

In [6]:
n2 = np.array(n2)
n2.shape

(6,)

In [7]:
n1

array([u'axis0', u'axis1', u'block0_items', u'block0_values',
       u'block1_items', u'block1_values'], 
      dtype='<U13')

hf_1 = h5py.File('frenet.h5', 'w')
hf_2 = h5py.File('kappa.h5', 'w')

hf_1.create_dataset('dataset_1', data=n1)
hf_2.create_dataset('dataset_2', data=n2)
#<HDF5 dataset "dataset_2": shape (1000, 200), type "<f8">
#All we need to do now is close the file, which will write all of our work to disk.

hf.close()

In [17]:
n1(['axis0'])

TypeError: 'numpy.ndarray' object is not callable

In [18]:
n2

array([u'axis0', u'axis1', u'block0_items', u'block0_values',
       u'block1_items', u'block1_values'], 
      dtype='<U13')

In [19]:
type(n2)

numpy.ndarray

In [21]:
type(n2[0])

numpy.unicode_

In [3]:
reread = pd.read_hdf('kappa_debug_log.h5', 'kappa_log')

ImportError: HDFStore requires PyTables, "No module named 'tables'" problem importing

In [12]:
read = pd.read_hdf(n1)

NotImplementedError: Support for generic buffers has not been implemented.